<a href="https://colab.research.google.com/github/Priyank-Vaidya/AI-Judge/blob/main/AI_Judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

!pip install -q -U google-genai

import os
import json
import random
from google import genai
from google.genai import types
from google.colab import userdata

client = genai.Client(api_key=api_key)


# --- THE PROMPT (The "Brain") ---
SYSTEM_PROMPT = """
You are the AI Judge for a game of "Rock-Paper-Scissors-PLUS".
Your goal is to evaluate user moves, enforce rules, and determine the round winner based on the provided game state.

### Rules
1. **Standard Moves:** Rock, Paper, Scissors.
   - Rock beats Scissors.
   - Scissors beats Paper.
   - Paper beats Rock.
2. **Special Move:** Bomb.
   - Bomb beats Rock, Paper, and Scissors.
   - Bomb vs Bomb results in a Draw.
   - **Constraint:** A user can use the "Bomb" only ONCE per entire game session.
3. **Validity:**
   - If the input is ambiguous, nonsense, or unrelated, mark as "UNCLEAR".
   - If the user tries to use "Bomb" but `bomb_already_used` is true, mark as "INVALID" (Illegal Move).
4. **Scoring:**
   - Unclear or Invalid moves "waste the turn" (User loses the round automatically).

### Input Context
You will be provided with:
1. `User Input`: The raw text from the player.
2. `Bot Move`: The move the opponent played (pre-generated).
3. `Bomb Already Used`: A boolean indicating if the user has already spent their special move.

### Process (Chain of Thought)
1. **Intent Extraction:** Map the user's raw text to a canonical move (rock, paper, scissors, bomb). Handle synonyms (e.g., "stone" -> "rock"). If no mapping exists, label as "unknown".
2. **Validation:** Check if the move is allowed.
   - If "unknown" -> Status: UNCLEAR.
   - If "bomb" AND `bomb_already_used` is True -> Status: INVALID.
   - Otherwise -> Status: VALID.
3. **Adjudication:** Compare User Move vs Bot Move to determine the winner (User, Bot, or Draw).
   - If Status is UNCLEAR or INVALID, Bot wins (Turn wasted).
4. **Reasoning:** Generate a short, punchy explanation for the player.

### Output
You must respond with a strictly valid JSON object:
{
  "interpreted_move": "rock" | "paper" | "scissors" | "bomb" | null,
  "status": "VALID" | "INVALID" | "UNCLEAR",
  "did_user_use_bomb_this_turn": boolean,
  "winner": "user" | "bot" | "draw",
  "explanation": "Brief explanation of why the user won/lost/drew."
}
"""

class GameState:
    def __init__(self):
        self.user_score = 0
        self.bot_score = 0
        self.bomb_used = False
        self.round = 1

def play_round(state, user_input):
    moves = ['rock', 'paper', 'scissors']
    bot_move = random.choice(moves)

    round_context = f"""
    User Input: "{user_input}"
    Bot Move: "{bot_move}"
    Bomb Already Used: {state.bomb_used}
    """
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            config=types.GenerateContentConfig(
                system_instruction=SYSTEM_PROMPT,
                response_mime_type="application/json"
            ),
            contents=[round_context]
        )

        decision = json.loads(response.text)

    except Exception as e:
        print(f"AI Referee Error: {e}")
        return

    if decision['did_user_use_bomb_this_turn']:
        state.bomb_used = True

    if decision['winner'] == 'user':
        state.user_score += 1
    elif decision['winner'] == 'bot':
        state.bot_score += 1

    print(f"\n--- Round {state.round} ---")
    print(f"You said: '{user_input}' -> Interpreted as: {decision['interpreted_move']}")
    print(f"Bot played: {bot_move}")
    print(f"Status: {decision['status']}")
    print(f"Result: {decision['explanation']}")
    print(f"Winner: {decision['winner'].upper()}")

    state.round += 1

def main():
    state = GameState()
    print("Welcome to AI Rock-Paper-Scissors-PLUS!")
    print("Rules: Standard RPS rules. You have ONE 'Bomb' that beats everything.")
    print("Type 'quit' to exit.")

    while True:
        try:
            user_input = input("\nEnter your move: ")
            if user_input.lower() in ['quit', 'exit']:
                break

            play_round(state, user_input)
            print(f"Score - You: {state.user_score} | Bot: {state.bot_score}")

        except KeyboardInterrupt:
            print("\nGame Over!")
            break

# Run the game
main()

Welcome to AI Rock-Paper-Scissors-PLUS!
Rules: Standard RPS rules. You have ONE 'Bomb' that beats everything.
Type 'quit' to exit.

Enter your move: I read the newspaper

--- Round 1 ---
You said: 'I read the newspaper' -> Interpreted as: None
Bot played: paper
Status: UNCLEAR
Result: Your move 'I read the newspaper' was unclear. You lose this round automatically.
Winner: BOT
Score - You: 0 | Bot: 1

Enter your move: I throw a heavy stone

--- Round 2 ---
You said: 'I throw a heavy stone' -> Interpreted as: rock
Bot played: scissors
Status: VALID
Result: Your rock smashes the bot's scissors!
Winner: USER
Score - You: 1 | Bot: 1

Enter your move: I deploy a Nuke

--- Round 3 ---
You said: 'I deploy a Nuke' -> Interpreted as: bomb
Bot played: rock
Status: VALID
Result: Your devastating Nuke easily overpowers the bot's Rock! You win this round!
Winner: USER
Score - You: 2 | Bot: 1

Enter your move: I play Paper

--- Round 4 ---
You said: 'I play Paper' -> Interpreted as: paper
Bot played: